# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
file_path = "CityWeather.csv"
weather_df = pd.read_csv(file_path)
weather_df.head()

,City Id,City Name,Temprature,Cloudiness,Wind,Latitude,Longitude,Humidity
0,2122090,Pevek,42.08,0,1.57,69.70,170.31,76
1,2156643,Mount Gambier,51.80,90,9.17,-37.83,140.77,93
2,2312249,Mushie,76.19,99,6.31,-3.02,16.90,79
3,5909514,Brooks,77.00,9,11.41,50.58,-111.89,42
4,3366880,Hermanus,48.00,97,1.99,-34.42,19.23,84


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humid = weather_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid,dissipating=False, max_intensity=10,point_radius=1)
                                 
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.

In [28]:
weather_df

,City Id,City Name,Temprature,Cloudiness,Wind,Latitude,Longitude,Humidity
0,2122090,Pevek,42.08,0,1.57,69.70,170.31,76
1,2156643,Mount Gambier,51.80,90,9.17,-37.83,140.77,93
2,2312249,Mushie,76.19,99,6.31,-3.02,16.90,79
3,5909514,Brooks,77.00,9,11.41,50.58,-111.89,42
4,3366880,Hermanus,48.00,97,1.99,-34.42,19.23,84
...,...,...,...,...,...,...,...,...
539,1686858,Siayan,85.53,17,3.76,8.27,123.10,66
540,2172880,Byron Bay,68.34,0,16.11,-28.65,153.62,49
541,3414079,Olafsvik,52.16,100,15.84,64.89,-23.71,92
542,1258269,Rāzole,75.79,100,10.11,16.48,81.83,91


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
res = weather_df[(weather_df['Temprature'] < 80.00) & (weather_df['Temprature'] > 70.00)]
hotel_df = res[(res['Wind'] < 10.00) & (res['Cloudiness'] == 0)].reset_index(drop=True)
loc_list = hotel_df[['Latitude', 'Longitude']].values.tolist()

In [30]:
hotel_names = []
for i in loc_list:
    target_coordinates = f'{i[0]},{i[1]}'
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    if len(response["results"]) >1:
        name = response["results"][0]["name"]
        hotel_names.append(name)
    else:
        hotel_names.append('')
    

In [31]:
hotel_df

,City Id,City Name,Temprature,Cloudiness,Wind,Latitude,Longitude,Humidity
0,4035715,Avarua,77.00,0,9.17,-21.21,-159.78,94
1,86049,Jalu,74.70,0,6.64,29.03,21.55,44
2,2985947,Porto-Vecchio,71.51,0,5.82,41.59,9.28,94
3,3166034,Sulmona,72.34,0,4.70,42.05,13.93,88
4,3984997,San Quintín,76.05,0,6.76,30.48,-115.95,65
5,2142316,Yeppoon,73.40,0,8.05,-23.13,150.73,33
6,293842,Kiryat Gat,73.69,0,3.06,31.61,34.76,80
7,88533,Awjilah,75.07,0,6.46,29.11,21.29,43
8,2219235,Ubari,77.02,0,7.16,26.59,12.78,27
9,264371,Athens,78.39,0,4.70,37.98,23.72,50


In [32]:
print(hotel_names)

['Paradise Inn', 'Jalu Hotel', 'Hôtel Le Goéland - Casa Santini', 'Hotel Santacroce', 'Old Mill Hotel', 'Villa Mar Colina', 'רגע בנחת', '', 'فندق أوباري', 'Novotel Athenes', 'O Carinhanha Hotel', 'Hotel Pousada dos Sonhos', '', 'Holiday Inn Express & Suites Lenoir City (Knoxville Area)']


In [33]:
hotel_df['Hotel Name'] = hotel_names


In [34]:
#dropped row which has empty hotel info (row index 7 and 12)
hotel_df.drop(hotel_df.index[[7,12]],inplace=True)

hotel_df

,City Id,City Name,Temprature,Cloudiness,Wind,Latitude,Longitude,Humidity,Hotel Name
0,4035715,Avarua,77.00,0,9.17,-21.21,-159.78,94,Paradise Inn
1,86049,Jalu,74.70,0,6.64,29.03,21.55,44,Jalu Hotel
2,2985947,Porto-Vecchio,71.51,0,5.82,41.59,9.28,94,Hôtel Le Goéland - Casa Santini
3,3166034,Sulmona,72.34,0,4.70,42.05,13.93,88,Hotel Santacroce
4,3984997,San Quintín,76.05,0,6.76,30.48,-115.95,65,Old Mill Hotel
5,2142316,Yeppoon,73.40,0,8.05,-23.13,150.73,33,Villa Mar Colina
6,293842,Kiryat Gat,73.69,0,3.06,31.61,34.76,80,רגע בנחת
8,2219235,Ubari,77.02,0,7.16,26.59,12.78,27,فندق أوباري
9,264371,Athens,78.39,0,4.70,37.98,23.72,50,Novotel Athenes
10,3466950,Carinhanha,72.37,0,6.46,-14.30,-43.77,47,O Carinhanha Hotel


In [35]:
country = ['CK','LY','FRA', 'IT', 'MX', 'AU', 'IL', 'LY', 'GR', 'BR', 'BR', 'US']
hotel_df['Country'] = country
hotel_df

,City Id,City Name,Temprature,Cloudiness,Wind,Latitude,Longitude,Humidity,Hotel Name,Country
0,4035715,Avarua,77.00,0,9.17,-21.21,-159.78,94,Paradise Inn,CK
1,86049,Jalu,74.70,0,6.64,29.03,21.55,44,Jalu Hotel,LY
2,2985947,Porto-Vecchio,71.51,0,5.82,41.59,9.28,94,Hôtel Le Goéland - Casa Santini,FRA
3,3166034,Sulmona,72.34,0,4.70,42.05,13.93,88,Hotel Santacroce,IT
4,3984997,San Quintín,76.05,0,6.76,30.48,-115.95,65,Old Mill Hotel,MX
5,2142316,Yeppoon,73.40,0,8.05,-23.13,150.73,33,Villa Mar Colina,AU
6,293842,Kiryat Gat,73.69,0,3.06,31.61,34.76,80,רגע בנחת,IL
8,2219235,Ubari,77.02,0,7.16,26.59,12.78,27,فندق أوباري,LY
9,264371,Athens,78.39,0,4.70,37.98,23.72,50,Novotel Athenes,GR
10,3466950,Carinhanha,72.37,0,6.46,-14.30,-43.77,47,O Carinhanha Hotel,BR


In [36]:
# Renaming columns headers and adding new columns to the DataFrame

hotel_df.rename(columns={'City Name': 'City'}, inplace=True)
hotel_df.rename(columns={'Latitude': 'Lat', 'Longitude': 'Lng'}, inplace=True)

hotel_df

,City Id,City,Temprature,Cloudiness,Wind,Lat,Lng,Humidity,Hotel Name,Country
0,4035715,Avarua,77.00,0,9.17,-21.21,-159.78,94,Paradise Inn,CK
1,86049,Jalu,74.70,0,6.64,29.03,21.55,44,Jalu Hotel,LY
2,2985947,Porto-Vecchio,71.51,0,5.82,41.59,9.28,94,Hôtel Le Goéland - Casa Santini,FRA
3,3166034,Sulmona,72.34,0,4.70,42.05,13.93,88,Hotel Santacroce,IT
4,3984997,San Quintín,76.05,0,6.76,30.48,-115.95,65,Old Mill Hotel,MX
5,2142316,Yeppoon,73.40,0,8.05,-23.13,150.73,33,Villa Mar Colina,AU
6,293842,Kiryat Gat,73.69,0,3.06,31.61,34.76,80,רגע בנחת,IL
8,2219235,Ubari,77.02,0,7.16,26.59,12.78,27,فندق أوباري,LY
9,264371,Athens,78.39,0,4.70,37.98,23.72,50,Novotel Athenes,GR
10,3466950,Carinhanha,72.37,0,6.46,-14.30,-43.77,47,O Carinhanha Hotel,BR


In [37]:
# Add marker layer ontop of heat map

locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humid = weather_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid,dissipating=False, max_intensity=10,point_radius=1)
                                 
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(marker_layer)


fig

Figure(layout=FigureLayout(height='420px'))